# Load Silver Tables


In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

silver_customers = spark.table("silver_customers")
silver_agents    = spark.table("silver_agents")
silver_policies  = spark.table("silver_policies")
silver_premiums  = spark.table("silver_premiums")
silver_claims    = spark.table("silver_claims")

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 3, Finished, Available, Finished)

# DimDate

In [2]:
min_date = (
    silver_premiums.select(F.min("payment_date").alias("d")).unionByName(
        silver_claims.select(F.min("claim_date").alias("d"))
    ).select(F.min("d").alias("min_d")).collect()[0]["min_d"]
)

max_date = (
    silver_premiums.select(F.max("payment_date").alias("d")).unionByName(
        silver_claims.select(F.max("claim_date").alias("d"))
    ).select(F.max("d").alias("max_d")).collect()[0]["max_d"]
)

print("DimDate range:", min_date, "to", max_date)

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 4, Finished, Available, Finished)

DimDate range: 2014-01-01 to 2029-11-12


In [3]:
dim_date = (
    spark.sql(f"SELECT explode(sequence(to_date('{min_date}'), to_date('{max_date}'), interval 1 day)) as date")
    .withColumn("date_key", F.date_format("date", "yyyyMMdd").cast("int"))
    .withColumn("day", F.dayofmonth("date"))
    .withColumn("month", F.month("date"))
    .withColumn("month_name", F.date_format("date", "MMMM"))
    .withColumn("quarter", F.quarter("date"))
    .withColumn("year", F.year("date"))
    .withColumn("week_of_year", F.weekofyear("date"))
    .withColumn("is_weekend", F.when(F.dayofweek("date").isin([1,7]), F.lit(True)).otherwise(F.lit(False)))
)

display(dim_date)

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 31524bdf-a82e-4668-869b-4f1fd9bad0dd)

In [4]:
(dim_date.write.format("delta")
 .mode("overwrite")
 .saveAsTable("gold_dim_date"))

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 6, Finished, Available, Finished)

# DimCustomer

In [5]:
w = Window.orderBy("customer_id")

dim_customer = (
    silver_customers
    .dropDuplicates(["customer_id"])
    .withColumn("customer_key", F.row_number().over(w))
    .select(
        "customer_key",
        "customer_id",
        "full_name",
        "gender",
        "age",
        "city",
        "state",
        "risk_segment",
        "join_date",
        "preferred_agent_id"
    )
)

display(dim_customer)

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f988ffce-f8c9-41d4-9f32-6988a580fc62)

In [6]:
(dim_customer.write.format("delta")
 .mode("overwrite")
 .saveAsTable("gold_dim_customer"))

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 8, Finished, Available, Finished)

# DimAgent

In [7]:
w = Window.orderBy("agent_id")

dim_agent = (
    silver_agents
    .dropDuplicates(["agent_id"])
    .withColumn("agent_key", F.row_number().over(w))
    .select(
        "agent_key",
        "agent_id",
        "agent_name",
        "region",
        "channel",
        "join_date"
    )
)

display(dim_agent)

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 18612408-b417-4ba8-a4ff-30e30aba6a06)

In [8]:
(dim_agent.write.format("delta")
 .mode("overwrite")
 .saveAsTable("gold_dim_agent"))

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 10, Finished, Available, Finished)

# DimPolicy

In [9]:
w = Window.orderBy("policy_id")

dim_policy = (
    silver_policies
    .dropDuplicates(["policy_id"])
    .withColumn("policy_key", F.row_number().over(w))
    .withColumn("policy_start_date_key", F.date_format("start_date", "yyyyMMdd").cast("int"))
    .withColumn("policy_end_date_key", F.date_format("end_date", "yyyyMMdd").cast("int"))
    .select(
        "policy_key",
        "policy_id",
        "customer_id",
        "agent_id",
        "policy_type",
        "plan_name",
        "policy_status",
        "premium_frequency",
        "sum_insured",
        "deductible",
        "policy_start_date_key",
        "policy_end_date_key"
    )
)

display(dim_policy)

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f9829186-b22f-4e16-a617-3e9bacb4a40d)

In [10]:
(dim_policy.write.format("delta")
 .mode("overwrite")
 .saveAsTable("gold_dim_policy"))

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 12, Finished, Available, Finished)

# FactPremiumPayments

In [11]:
fact_premium = (
    silver_premiums
    .withColumn("date_key", F.date_format("payment_date", "yyyyMMdd").cast("int"))
    .join(dim_policy.select("policy_id", "policy_key"), on="policy_id", how="left")
    .join(dim_customer.select("customer_id", "customer_key"), on="customer_id", how="left")
    .join(dim_agent.select("agent_id", "agent_key"), on="agent_id", how="left")
    .select(
        "payment_id",
        "date_key",
        "policy_key",
        "customer_key",
        "agent_key",
        "premium_amount",
        "payment_status",
        "payment_method",
        "currency"
    )
)

display(fact_premium)

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 08804c0b-58a6-4919-b2e9-8b92e60d2d27)

In [12]:
(fact_premium.write.format("delta")
 .mode("overwrite")
 .saveAsTable("gold_fact_premium_payments"))

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 14, Finished, Available, Finished)

# FactClaims

In [13]:
fact_claims = (
    silver_claims
    .withColumn("date_key", F.date_format("claim_date", "yyyyMMdd").cast("int"))
    .join(dim_policy.select("policy_id", "policy_key"), on="policy_id", how="left")
    .join(dim_customer.select("customer_id", "customer_key"), on="customer_id", how="left")
    .join(dim_agent.select("agent_id", "agent_key"), on="agent_id", how="left")
    .select(
        "claim_id",
        "date_key",
        "policy_key",
        "customer_key",
        "agent_key",
        "claim_status",
        "claim_reason",
        "claim_amount",
        "approved_amount",
        "days_to_settle"
    )
)

display(fact_claims)

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 27450772-d646-4368-80ae-84212f655285)

In [14]:
(fact_claims.write.format("delta")
 .mode("overwrite")
 .saveAsTable("gold_fact_claims"))

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 16, Finished, Available, Finished)

# Quality Check

In [15]:
print("=== GOLD DQ CHECKS ===")

print("Premium fact null policy_key:", fact_premium.filter("policy_key IS NULL").count())
print("Premium fact null customer_key:", fact_premium.filter("customer_key IS NULL").count())
print("Premium fact null agent_key:", fact_premium.filter("agent_key IS NULL").count())
print("Premium fact null date_key:", fact_premium.filter("date_key IS NULL").count())

print("Claims fact null policy_key:", fact_claims.filter("policy_key IS NULL").count())
print("Claims fact null customer_key:", fact_claims.filter("customer_key IS NULL").count())
print("Claims fact null agent_key:", fact_claims.filter("agent_key IS NULL").count())
print("Claims fact null date_key:", fact_claims.filter("date_key IS NULL").count())

StatementMeta(, 5baf0845-170b-4369-b01f-b0852d0b773a, 17, Finished, Available, Finished)

=== GOLD DQ CHECKS ===
Premium fact null policy_key: 0
Premium fact null customer_key: 0
Premium fact null agent_key: 0
Premium fact null date_key: 0
Claims fact null policy_key: 0
Claims fact null customer_key: 0
Claims fact null agent_key: 0
Claims fact null date_key: 0
